In [3]:
import re
import pandas as pd

In [6]:
string = '''
manufacture.produce_task as pt
SELECT
	a.*,
	a.实际合格产出 + a.实际不合格产出 AS 实际总产出 
FROM
	(
	SELECT
		pt.process_name 工序名称,
		pt.process_code 工序编号,
		pt.process_name 工序简称,
		SUM( pt.amount_product_planned ) 计划数,
		( CASE WHEN ISNULL( SUM( shr.amount_hold ) ) THEN 0 ELSE SUM( shr.amount_hold ) END ) 实际合格产出,
		( CASE WHEN ISNULL( SUM( shur.amount_diff ) ) THEN 0 ELSE SUM( shur.amount_diff ) END ) 实际不合格产出 
	FROM
		( SELECT * FROM manufacture.produce_task WHERE org_id = 610009 ) pt
		INNER JOIN (
		SELECT
			shr.task_id,
			shr.org_id,
			SUM( shr.amount_hold ) amount_hold,
			created_at 
		FROM
			manufacture.scan_hold_record shr 
		WHERE
			DATE( created_at ) = CURDATE( ) 
		GROUP BY
			shr.task_id 
		) shr ON shr.task_id = pt.id 
		AND shr.org_id = pt.org_id
		LEFT JOIN ( SELECT org_id, task_id, sum( amount_diff ) amount_diff, created_at FROM manufacture.scan_hold_unqualified_record WHERE DATE( created_at ) = CURDATE( ) GROUP BY task_id ) AS shur ON shur.org_id = pt.org_id 
		AND shur.task_id = pt.id 
	WHERE
		pt.org_id = 610009 
	GROUP BY
	pt.process_code 
	) a'''

In [7]:
pat1 = re.compile(r'FROM +((?!\()\w+?)\.')
pat2 = re.compile(r'FROM\n\t*((?!\()\w+?)\.')
pat3 = re.compile(r'LEFT JOIN +((?!\()\w+?)\.')
pat3_AS = re.compile(r'LEFT JOIN +AS ((?!\()\w+?)\.')

pat1l = re.compile(r'from +((?!\()\w+?)\.')
pat2l = re.compile(r'from\n*\t*((?!\()\w+?)\.')
pat3l = re.compile(r'left join +((?!\()\w+?)\.')
pat3l_as = re.compile(r'left join +as ((?!\()\w+?)\.')
r = list(set(re.findall(pat1,string) + re.findall(pat2,string) + re.findall(pat1l,string) + re.findall(pat2l,string) + 
       re.findall(pat3,string) + re.findall(pat3_AS,string) + re.findall(pat3l,string)+ re.findall(pat3l_as,string)))
#提取库名

print ('库  :  表','\n')

for item in r:
    p = re.compile(item+r'\.([^ \n](?!\{)(?!\$)\w+)')
    temp = list(set(re.findall(p,string)))
    temp.sort()
    # 提取表名
    print (item,':',temp,'\n')
    temp.append('-------------------------')
    
    item = '库:'+item
    df2 = pd.DataFrame()
    df2 = pd.DataFrame(columns = [item], data = temp)
    df2.to_csv('/Users/data/desktop/table.csv', mode = 'a', index = False)
    # 输出「库：表」
    temp = temp[:-1]
    
    for table in temp:
        p_1 = re.compile(r'\.'+table+r' +((?!where)(?!WHERE)(?!as)(?!AS)\w+)')
        p_2 = re.compile(r'\.'+table+r' +AS (\w+)')
        p_2l = re.compile(r'\.'+table+r' +as (\w+)')
        alias = list(set(re.findall(p_1,string)+re.findall(p_2,string)+re.findall(p_2l,string)))
        if alias:
            print ('\t',table,':',alias,'\n')
        else:
            print ('\n\t',table,':未重命名','\n')
        #提取表的重命名
        
        ziduan = []
        
        if alias:
            for i in alias:
                p_3 = re.compile(r' '+i+'\.(\w+)')
                p_4 = re.compile(r', *'+i+'\.(\w+)')
                p_5 = re.compile(r'= *'+i+'\.(\w+)')
                p_6 = re.compile(r'\( *'+i+'\.(\w+)')
                p_7 = re.compile(r'\n *\t*'+i+'\.(\w+)')
                ziduan = list(set(ziduan+list(set(re.findall(p_3,string)+re.findall(p_4,string)
                                       +re.findall(p_5,string)+re.findall(p_6,string)+re.findall(p_7,string)))))
                ziduan = list(set(list(map(lambda x: x.lower(), ziduan ))))
            
            ziduan.sort() 
            #将字段按首字母进行排序
            
            print ('\t\t共',len(ziduan),'个字段:',ziduan)
            ziduan.append('-------------------------')
            
            table = '表:'+table
            df = pd.DataFrame()
            df = pd.DataFrame(columns = [table], data = ziduan)
            df.to_csv('/Users/data/desktop/ziduan.csv', mode = 'a', index = False)
            #提取字段并输出

库  :  表 

manufacture : ['produce_task', 'scan_hold_record', 'scan_hold_unqualified_record'] 

	 produce_task : ['pt'] 

		共 5 个字段: ['amount_product_planned', 'id', 'org_id', 'process_code', 'process_name']
	 scan_hold_record : ['shr'] 

		共 3 个字段: ['amount_hold', 'org_id', 'task_id']

	 scan_hold_unqualified_record :未重命名 

